In [55]:
import pandas as pd
import numpy as np
import requests
import json
import os
import collections
collections.Callable = collections.abc.Callable
from bs4 import BeautifulSoup

In [7]:
propublica_token=os.environ['propublica_token']

useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']
useragent

'python-requests/2.28.1'

In [8]:
headers={'X-API-Key': propublica_token,
         'User-Agent': useragent,
        'From': 'czj9zj@virginis.edu'}

## Goal: Get the text of all bills sponsored by Bob Good in the 117th Congress
### Step 1: Get bob Good's ID number from the propublica members API

In [11]:
root = "https://api.propublica.org"
congress="117"
chamber="house"
endpoint = "/congress/v1/{congress}/{chamber}/members.json".format(congress=congress,chamber=chamber)

r = requests.get(root+endpoint,headers=headers)

myjson = json.loads(r.text)
membersdf = pd.json_normalize(myjson,record_path=["results","members"])
membersdf.head(3).T

,0,1,2
id,A000370,A000055,A000371
title,Representative,Representative,Representative
short_title,Rep.,Rep.,Rep.
api_uri,https://api.propublica.org/congress/v1/members...,https://api.propublica.org/congress/v1/members...,https://api.propublica.org/congress/v1/members...
first_name,Alma,Robert,Pete
middle_name,None,B.,None
last_name,Adams,Aderholt,Aguilar
suffix,None,None,None
date_of_birth,1946-05-27,1965-07-22,1979-06-19
gender,F,M,M


In [14]:
bobgood = membersdf.query("last_name == 'Good'")

In [16]:
bobgoodid = bobgood.reset_index()['id'][0]

## Step 2: Use BG's ID to query the bills API

In [22]:
endpoint = '/congress/v1/members/{memberid}/bills/{billtype}.json'.format(memberid=bobgoodid,billtype='introduced')

r = requests.get(root+endpoint, headers = headers)

myjson = json.loads(r.text)

bgbills1 = pd.json_normalize(myjson, record_path = ['results','bills'])

In [25]:
endpoint = '/congress/v1/members/{memberid}/bills/{billtype}.json'.format(memberid=bobgoodid,billtype='introduced')

r = requests.get(root+endpoint, headers = headers, params = {'offset':20})

myjson = json.loads(r.text)

bgbills2 = pd.json_normalize(myjson, record_path = ['results','bills'])

In [28]:
bgbills = pd.concat([bgbills1,bgbills2],ignore_index=True)
bgbills.head(3).T

,0,1,2
congress,117,117,117
bill_id,hres389-117,hr3045-117,hr2879-117
bill_type,hres,hr,hr
number,H.RES.389,H.R.3045,H.R.2879
bill_uri,https://api.propublica.org/congress/v1/117/bil...,https://api.propublica.org/congress/v1/117/bil...,https://api.propublica.org/congress/v1/117/bil...
title,"Designating the week beginning November 8, 202...",To provide limitations for Federal agencies en...,To prevent a fiscal crisis by enacting legisla...
short_title,"Designating the week beginning November 8, 202...",No Regulation Through Litigation Act of 2021,Nickel Plan Act
sponsor_title,Rep.,Rep.,Rep.
sponsor_id,G000595,G000595,G000595
sponsor_name,Robert Good,Robert Good,Robert Good


In [45]:
urltoscrape = bgbills['congressdotgov_url'][10] + '/text'
urltoscrape

'https://www.congress.gov/bill/117th-congress/house-bill/991/text'

In [57]:
r = requests.get(urltoscrape, headers = {'User_Agent': useragent, 'From': 'czj9zj@virginia.edu'})

myhtml = BeautifulSoup(r.text, 'html.parser')

In [64]:
myhtml.find_all('p', "lbexIndent", style="text-align:left")[0]S.text

' Be it enacted by the Senate and House of Representatives of the\n\t\tUnited States of America in Congress assembled,  SECTION 1.  Short title.This Act may be cited as the “Preserving Safety Net Integrity Act of 2021”. SEC. 2.  Requiring implementation of final rule.Notwithstanding any other provision of law, the Secretary of Homeland Security may not take any action to rescind, revise, or suspend the final rule, entitled “Inadmissibility on Public Charge Grounds” (85 Fed. Reg. 41292, August 14, 2019). '